In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import networkx as nx
import scipy.sparse as sp
from random import randint
from sklearn.decomposition import PCA
#download PCA
import time
from gensim.models import Word2Vec
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import nltk
import json
from catboost import CatBoostRegressors
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import hstack

import re
import lightgbm as lgb
from nltk.corpus import stopwords

# Deepwalk on matrix structure

In [4]:
# exact code from the last seminar
def random_walk(G, node, walk_length):
    # Simulates a random walk of length "walk_length" starting from node "node"

    #Please insert your code for Task 1 here
    walk = [node]
    for i in range(walk_length):
        neighbors =  list(G.neighbors(walk[i]))
        walk.append(neighbors[randint(0,len(neighbors)-1)])

    walk = [str(node) for node in walk]
    return walk

def generate_walks(G, num_walks, walk_length):
    # Runs "num_walks" random walks from each node

    walks = []

    #Please insert your code for Task 1 here
    for i in range(num_walks):
        permuted_nodes = np.random.permutation(G.nodes())
        for node in permuted_nodes:
            walks.append(random_walk(G, node, walk_length))

    return walks

def deepwalk(G, num_walks, walk_length, n_dim):
    # Simulates walks and uses the Skipgram model to learn node representations

    print("Generating walks")
    walks = generate_walks(G, num_walks, walk_length)

    print("Training word2vec")
    
    #Please insert your code for Task 2 here
    model = Word2Vec(vector_size=n_dim, window=8, min_count=0, sg=1, workers=8, hs=1)
    
    
    model.build_vocab(walks)
    model.train(walks, total_examples=model.corpus_count, epochs=5)

    return model

In [3]:
# read training data
df_train = pd.read_csv('train.csv', dtype={'author': np.int64, 'hindex': np.float32})
n_train = df_train.shape[0]

# read test data
df_test = pd.read_csv('test.csv', dtype={'author': np.int64})
n_test = df_test.shape[0]

# load the graph    
G = nx.read_edgelist('coauthorship.edgelist', delimiter=' ', nodetype=int)
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges() 
print('Number of nodes:', n_nodes)
print('Number of edges:', n_edges)
core_number = nx.core_number(G)
pr = nx.pagerank(G)
for el in pr:
  pr[el] = pr[el]*1000000

Number of nodes: 217801
Number of edges: 1718164


In [ ]:
start = time.time()
n_dim = 50
n_walks = 5
walk_length = 8
model = deepwalk(G, n_walks, walk_length, n_dim) 
print(time.time()-start, "time for deep walk")


In [ ]:
# Calculating on n_dim = 50, n_walks = 5, walk_length = 8  413 seconds
# Calculating on n_dim = 64, n_walks = 5, walk_length = 14  585 seconds
# Calculating on n_dim = 80, n_walks = 7, walk_length = 20  585 seconds

In [33]:
DeepWalk_embeddings = np.empty(shape=(df_train.shape[0]+df_test.shape[0], n_dim))
for idx,node in enumerate(G.nodes()):
    DeepWalk_embeddings[idx,:] = model.wv[str(node)]

nodes_graph = list(G.nodes())
vec_embed = {} # nodes corresponds to deepwalks_embeddings
for el in range(len(nodes_graph)):
    vec_embed[nodes_graph[el]] = DeepWalk_embeddings[el] 

In [4]:
X_train = np.zeros((n_train, 5))
y_train = np.zeros(n_train)
for i,row in df_train.iterrows():
    node = row['author']
    X_train[i, 0] = node
    X_train[i,1] = G.degree(node)
    X_train[i,2] = core_number[node]
    X_train[i,3] = pr[node]
    y_train[i] = row['hindex']

# create test matrix
X_test = np.zeros((n_test, 5))
for i,row in df_test.iterrows():
    node = row['author']
    X_test[i, 0] = node
    X_test[i,1] = G.degree(node)
    X_test[i,2] = core_number[node]
    X_test[i,3] = pr[node]
    
X_train_test = np.concatenate((X_train, X_test), axis = 0)

In [34]:
arr_vec_embed = []
for i,row in df_train.iterrows():
    node = row['author']
    X_train[i, 0] = node
    arr_vec_embed.append(vec_embed[node])
for i,row in df_test.iterrows():
    node = row['author']
    X_test[i, 0] = node
    arr_vec_embed.append(vec_embed[node])
    
    
df_arr= pd.DataFrame(data=arr_vec_embed) #dataframe of deepwalk embeddings

vec_embed = {}
arr_vec_embed = []
pr = []
model = 0
DeepWalk_embeddings = 0

In [35]:
#df_arr.to_pickle('deep_walk50')

In [239]:
from scipy.stats.stats import pearsonr   
  
pearsonr(X_train[:, 1],y_train) # correlation of degree

(0.4063264559353831, 0.0)

In [240]:
pearsonr(X_train[:, 2],y_train) # corrrelation of core_number

(0.3590667720340064, 0.0)

In [241]:
pearsonr(X_train[:, 3],y_train) #pagerank correlation

(0.40216429459635095, 0.0)

In [245]:
pearsonr(np.squeeze(mass_1[0:y_train.shape[0]]),y_train) # for average h_index

(0.3631428505021522, 0.0)

# Reading data from files

In [5]:
import time
start = time.time()
with open('abstracts.txt') as f:
    lines1 = f.readlines()
arr2 = []
indeces = []
for elem in range(0, len(lines1)):
    if elem % 100000 == 0:
        print(elem)
    mass = lines1[elem]
    indeces.append(int(mass.split('----')[0]))
    j = mass.split('----')[1]
    try:
        d = json.loads(j)
        mass1 = d['InvertedIndex']
        dl = d['IndexLength']
        arr1 = np.array(['ookdfnjdfmkdf' for _ in range(dl)])
        for el in mass1:
            for el1 in mass1[el]:
                arr1[el1] = el
    #arr2.append(arr1)
        arr2.append(' '.join(arr1))
    except ValueError:
        arr2.append(' ')
print(time.time()-start)

start = time.time()
abstracts = pd.DataFrame(index = indeces, data =arr2)
abstracts = abstracts.reset_index(level = [0])
abstracts = abstracts.set_axis(['paper_id', 'article'], axis='columns', inplace=False)
#abstracts.to_pickle('abstracts_table') # to save file
#abstracts = pd.read_pickle('abstracts_table') # to read file

0
100000
200000
300000
400000
500000
600000
119.17308807373047


In [6]:
d = {}
for el in range(len(indeces)):
    d[indeces[el]] = arr2[el]

In [7]:
with open('author_papers.txt') as f:
    lines1 = f.readlines()
    
indic = []
ids = []
d_num_papers = {}
arr = []
for i in range(0,len(lines1) ):
    cur_line = lines1[i].split(':')
    indic.append(int(cur_line[0]))
    d_num_papers[int(cur_line[0])] = len(cur_line[1].split('\n')[0].split('-'))
    arr.append(len(cur_line[1].split('\n')[0].split('-')))
    ids.append(cur_line[1].split('\n')[0].split('-'))

In [507]:
d1 = {'id':[1036332], 'article1': [1510273386], 'article2': [1827736641], 'article3':[1588673897], 'article4':[2252711322], 'article5':[2123653597]}
df_articles = pd.DataFrame(data=d1)

for i in range(0, len(ids)):
    cur_arr = np.zeros(6)
    cur_arr[0] = indic[i] 
    for j in range(len(ids[i])):
        #new_row = {'id':indic[i], 'articles':ids[i][j]} 
        cur_arr[j+1] = ids[i][j]
        
        
        #df_articles = df_articles.append(new_row, ignore_index=True)
        df_articles.loc[i] = cur_arr
    if i % 50000 == 0:
        print(i)

0


In [8]:
#df_articles = pd.read_pickle('articles_table')
df_articles.head()

,id,articles1,articles2,articles3,articles4,articles5
0,1036332.0,1.510273e+09,1.827737e+09,1.588674e+09,2.252711e+09,2.123654e+09
2,1101850.0,1.334590e+08,1.797197e+08,2.111788e+09,2.126489e+09,3.183900e+07
3,1336878.0,2.122092e+09,2.132110e+09,2.100272e+09,2.065673e+09,2.036414e+09
4,1515524.0,2.141828e+09,2.127086e+09,2.013548e+09,2.138530e+09,1.994864e+09
5,1606427.0,1.907725e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [16]:
#abstracts.to_pickle('articles_table') # to save file
#df_articles = pd.read_pickle('articles_table') # to read from file

In [9]:
id_paper_first = df_articles[['articles1']].to_numpy()
author_papers_id = df_articles[['id']].to_numpy()
id_paper_second = df_articles[['articles2']].to_numpy()
id_paper_third = df_articles[['articles3']].to_numpy()
id_paper_four = df_articles[['articles4']].to_numpy()
id_paper_five = df_articles[['articles5']].to_numpy()

In [10]:
d_author_id_article1= {}
d_author_id_article2= {}
d_author_id_article3= {}
d_author_id_article4= {}
d_author_id_article5= {}
for i in range(len(id_paper_first)):
    d_author_id_article1[author_papers_id[i][0]] = id_paper_first[i][0]
    d_author_id_article2[author_papers_id[i][0]] = id_paper_second[i][0]
    d_author_id_article3[author_papers_id[i][0]] = id_paper_third[i][0]
    d_author_id_article4[author_papers_id[i][0]] = id_paper_four[i][0]
    d_author_id_article5[author_papers_id[i][0]] = id_paper_five[i][0]

In [11]:
first_article = []
second_article = []
third_article = []
four_article = []
five_article = []
fl = 0
for i in range(X_train_test.shape[0]):
    try:
        el1 = d[d_author_id_article1[X_train_test[i, 0]]]
        el2 = d[d_author_id_article2[X_train_test[i, 0]]]
        el3 = d[d_author_id_article3[X_train_test[i, 0]]]
        el4 = d[d_author_id_article4[X_train_test[i, 0]]]
        el5 = d[d_author_id_article5[X_train_test[i, 0]]]
        first_article.append(el1)
        second_article.append(el2)
        third_article.append(el3)
        four_article.append(el4)
        five_article.append(el5)
        
    except KeyError:
        first_article.append(' ')
        second_article.append(' ')
        third_article.append(' ')
        four_article.append(' ')
        five_article.append(' ')
        fl += 1
    if i % 20000 == 0:
        print(i)
        
# erase data, because dict use much memory
d_author_id_article1= {}
d_author_id_article2= {}
d_author_id_article3= {}
d_author_id_article4= {}
d_author_id_article5= {}

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000


In [12]:
d_train_main  = {'id':X_train[:, 0], 'article_1': first_article[0:174241], 'article_2': second_article[0:174241], 'article_3':third_article[0:174241], 
                 'article_4':four_article[0:174241], 'article_5':five_article[0:174241]}
df_train_main = pd.DataFrame(data=d_train_main)

In [13]:
d_test_main = {'id':X_test[:, 0], 'article_1': first_article[174241:], 'article_2': second_article[174241:], 'article_3':third_article[174241:], 
                 'article_4':four_article[174241:], 'article_5':five_article[174241:]}
df_test_main = pd.DataFrame(data=d_test_main)


#df_train_main.to_pickle('train_article') # save to the file
#df_test_main.to_pickle('test_article')

#df_train_main = pd.read_pickle('train_article') #read from the file
#df_test_main = pd.read_pickle('test_article')

# Making features using word2vec

In [ ]:
stop_words = stopwords.words('english')

def clean_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"(@\[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sentence)
    #regular expression which deletes numbers 
    
    #return " ".join([word for word in sentence.split() if word not in (stop_words)])
    return sentence

In [ ]:

df_train_main['articles_tog'] = '' #sum all articles together
cur_mass = []
for i in range(df_train_main.shape[0]):
    cur_mass.append(df_train_main.loc[i].article_1 + df_train_main.loc[i].article_2+ df_train_main.loc[i].article_3+df_train_main.loc[i].article_4+df_train_main.loc[i].article_5)
    if i % 50000 == 0:
        print(i, "Train")

df_train_main['articles_tog'] = cur_mass
#df_train_main.to_pickle('train_article') # save results to df_train_main


df_test_main['articles_tog'] = '' #same for test
cur_mass = []
for i in range(df_test_main.shape[0]):
    cur_mass.append(df_test_main.loc[i].article_1 + df_test_main.loc[i].article_2+ df_test_main.loc[i].article_3+df_test_main.loc[i].article_4+df_test_main.loc[i].article_5)
    if i % 50000 == 0:
        print(i, "Test")

df_test_main['articles_tog'] = cur_mass
#df_test_main.to_pickle('test_article') 

In [ ]:
# articles_tog -> articles with summed all articles
start = time.time()
testing_clean = []

for i in range(df_train_main.shape[0]):
    sentence = clean_sentence(df_train_main.iloc[i].articles_tog)
    sentence = " ".join([word for word in sentence.split() if word not in (stop_words)])
    testing_clean.append(sentence)
    
for i in range(df_test_main.shape[0]):
    sentence = clean_sentence(df_test_main.iloc[i].articles_tog)
    sentence = " ".join([word for word in sentence.split() if word not in (stop_words)])
    testing_clean.append(sentence)
    
print(time.time()-start, "time for cleaning")

wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in testing_clean]

start = time.time()
# Set values for various parameters
feature_size = 100      # Word vector dimensionality  
window_context = 30     # Context window size                                                                                    
min_word_count = 1      # Minimum word count                        
sample = 1e-3           # Downsample setting for frequent words

w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size, 
                          window=window_context, min_count=min_word_count,
                          sample=sample, iter=50)
print(time.time()-start, "Time for word2vec model")

def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    
   
def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)


# get document level embeddings
start = time.time()
w2v_feature_array = averaged_word_vectorizer(corpus=tokenized_corpus, model=w2v_model,
                                             num_features=feature_size)
w2v_feature_array_1 = pd.DataFrame(w2v_feature_array) # Save data for word2vec features 
w2v_feature_array_1.to_pickle('w2v_feature_array_1')

# features_word2vec = pd.read_pickle('w2v_feature_array_1') #upload file 
print(time.time()-start)

# Creating new features

* 1 if author has 5 pages
* average h_index of neigbours for each author
* max_h_index of neighbours
* min h_index of neighbours

In [15]:
with open('author_papers.txt') as f:
    lines1 = f.readlines()
    
d_num_papers = {}
for i in range(0,len(lines1) ):
    cur_line = lines1[i].split(':')
    d_num_papers[int(cur_line[0])] = len(cur_line[1].split('\n')[0].split('-'))


In [16]:
new_features_5 = [] # feature 1 if author has 5 pages and 0 otherwise
for i in range(X_train_test.shape[0]):
    el = d_num_papers[X_train_test[i][0]]
    if el == 5:
        new_features_5.append(1)
    else:
        new_features_5.append(0)

In [17]:
d = {} # store for author -> h_index
for i in range(X_train.shape[0]):
    d[X_train[i][0]] = y_train[i]

mass_1 = [] # feature for calculating average h_index
start = time.time()
for i in range(X_train_test.shape[0]):
    
    neighb = list(G.neighbors(X_train_test[i][0]))
    ch = 0
    col = 0
    for neigh in neighb:
    
        if (neigh in d) == False:
            ch += 0
        else:
            ch += float(d[neigh])
            col += 1
    if col > 0:
        mass_1.append(ch/col)
    else:
        mass_1.append(0)
print(time.time()-start, "average h_index for neighbours")

mass_1 = np.array(mass_1)
mass_1 = mass_1.reshape(-1, 1)
new_features_5 = np.array(new_features_5)
new_features_5 = new_features_5.reshape(-1, 1)

3.3082919120788574 average h_index for neighbours


In [21]:
mass_maxi = []
mass_mini = []
start = time.time()
for i in range(X_train_test.shape[0]): 
    
    neighb = list(G.neighbors(X_train_test[i][0]))
    maxi = 0
    mini = 400
    for neigh in neighb:
        
        
        if (neigh in d) == False:
            maxi += 0
            mini += 0
        else:
            if  float(d[neigh]) > maxi:
                maxi = float(d[neigh])
        
            if  float(d[neigh]) < mini:
                mini = float(d[neigh])
                
            
    if mini == 400:
        mass_mini.append(0)
    else:
        mass_mini.append(mini)
    mass_maxi.append(maxi)

print(time.time()-start)

mass_maxi = np.array(mass_maxi)
mass_maxi = mass_maxi.reshape(-1, 1)
mass_mini = np.array(mass_mini)
mass_mini = mass_mini.reshape(-1, 1)

4.661071062088013


In [ ]:
features_word2vec = pd.read_pickle('w2v_feature_array_1') # file of word2vec features
df_arr = pd.read_pickle('deep_walk')
'''abstracts = pd.read_pickle('abstracts_table')
df_articles = pd.read_pickle('articles_table')
tfidf_vector = pd.read_pickle('author_tfidf_vector.txt')
'''

In [508]:
final_test = np.hstack((X_train[:, 1:4], mass_1[0:174241],  new_features_5[0:174241], features_word2vec[0:174241], df_arr[0:174241]))
final_test_test = np.hstack((X_test[:, 1:4],mass_1[174241:], new_features_5[174241:], features_word2vec[174241:],  df_arr[174241:]))

In [ ]:
train1, test1, y_train1, y_test1 = train_test_split(final_test, y_train, test_size=.33, random_state = 42)
start = time.time()
clf = CatBoostRegressor(n_estimators = 1000,learning_rate = 0.2,  depth = 5)
clf.fit(train1, y_train1)
y_pred = clf.predict(test1)
print(time.time()-start)

In [ ]:
'''# Analyze for predictions and their distribution compared to real model
 10 < x  11 to 15.5 37743 to 38812 
10 < x < 20 77 to 39     12711 to 8963
20 < x < 30 170 to 86.66  4127 to 3391 observations
30  < x < 50  277 to 210 mse 2137 to 2484
50 < x < 80 276 to 489 752 to 1049
> 80 30 to 160 500 vs 2800 '''


# Checking quality of the model

In [ ]:
n_estim = [100, 200, 300]
learning = [0.1,0.12]
num_leave = [20, 50, 100, 200]


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)
for el in n_estim:
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(final_test, y_train)):
        X_train, X_test = final_test[train_idx], final_test[test_idx]
        y_train1, y_test1 = y_train[train_idx], y_train[test_idx]

        model = lgb.LGBMRegressor(n_estimators = 500, learning_rate = 0.12, num_leaves = 200)
        model.fit(X_train, y_train1)
        preds = model.predict(X_test)
        cv_scores[idx] = mean_squared_error(y_test1, preds)
    print(cv_scores,np.mean(cv_scores),  el)

In [ ]:
#n_estimators
'''[53.53571033 54.50651916 53.60322133] 100
[51.77734503 52.61463367 51.70348592] 200
[50.42872934 50.96430724 50.22325336] 500'''

#lr
#[51.38879843 52.64495793 51.90799202 51.37096127 50.84715438] 51.63197280673071 0.12
#num_leaves

#[52.96982495 53.52256497 52.76881649 52.04011541 51.48920142] 52.558104647817274 20
#[51.3099745  51.78890643 50.82470602 50.65195368 50.0970291 ] 50.93451394471755 50
#[50.16356784 51.15144598 50.80201971 49.90393926 49.69891136] 50.343976829826204 100
#[50.06984358 51.40634118 50.22492043 50.51083878 49.07635478] 50.25765974824528 200

# Use there final model with best hyperparameters

In [ ]:
start = time.time()
reg = lgb.LGBMRegressor(n_estimators = 700,learning_rate = 0.12, num_leaves = 150)
reg.fit(final_test, y_train)
y_pred = reg.predict(final_test_test)

clf = CatBoostRegressor(n_estimators = 1000,learning_rate = 0.2,  depth = 7)
clf.fit(final_test, y_train)
y_pred1 = clf.predict(final_test_test)


clf1 = CatBoostRegressor(n_estimators = 1000,learning_rate = 0.2,  depth = 7, random_state = 54)
clf1.fit(final_test, y_train)
y_pred2 = clf1.predict(final_test_test)
print(time.time()-start)


In [155]:
y_pred1[y_pred1 <0] = 2 # 2 is better by checking local score on mse
y_pred2[y_pred2 <0] = 2
y_pred[y_pred <0] = 2


In [157]:
final_results = (y_pred2+y_pred1+y_pred)/3

# Saving results to the file

In [158]:
# Save the results to kaggle
df_test['hindex'] = pd.Series(np.round_(final_results, decimals=3))
df_test.loc[:,["author","hindex"]].to_csv('submission_second.csv', index=False)

In [ ]:
#What we tried, but didn't work

import time
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 200)
X_train_tfidf = tfidf.fit_transform(df_test_data['article'].to_numpy())
tdifd_arr = X_train_tfidf.toarray()

X_train_tfidf2 = tfidf.fit_transform(df_train_main['article_2'].to_numpy())
X_train_tfidf3 = tfidf.fit_transform(df_train_main['article_3'].to_numpy())
X_train_tfidf4 = tfidf.fit_transform(df_train_main['article_4'].to_numpy())
X_train_tfidf5 = tfidf.fit_transform(df_train_main['article_5'].to_numpy())

tdifd_arr = X_train_tfidf.toarray()
tdifd_arr2 = X_train_tfidf2.toarray()
tdifd_arr3 = X_train_tfidf3.toarray()
tdifd_arr4 = X_train_tfidf4.toarray()
tdifd_arr5 = X_train_tfidf5.toarray()

final_article1 = np.hstack((df_train_main[['degree', 'core']].to_numpy(), tdifd_arr,tdifd_arr2, tdifd_arr3, tdifd_arr4, tdifd_arr5 ))

